In [1]:
# ML Pipeline Notebook

## 1. Setup and Imports

import sys
from pathlib import Path

# Add project root to Python path
project_root = str(Path.cwd().parent)
if project_root not in sys.path:
    sys.path.append(project_root)

# Import your modules
import yaml
import pandas as pd
import numpy as np
from rich import print
from src.data.data_processing import DataAnalyzer, DataProcessor
from src.features.feature_engineering import FeatureMetadata
from src.models.model_training import ModelTrainer

# Load configuration
with open('configs/config.yml', 'r') as f:
    config = yaml.safe_load(f)


## 2. Data Loading and Initial Processing

# Initialize DataProcessor
data_processor = DataProcessor('configs/config.yml')

# Load raw data
raw_data_path = data_processor.config['data']['raw_data_path']
raw_data_file_name = data_processor.config['data']['raw_data_file_name']

# Load data and change columns data type
raw_data_df = data_processor.data_load(f'{raw_data_path}{raw_data_file_name}')

print('Initial raw data:')
display(raw_data_df.head())
print('\nInitial raw data shape:', raw_data_df.shape)


DATA LOADING: Infer data types based on the first 1000 rows

DATA LOADING: Data types inferred count
 object    3
int64     1
Name: count, dtype: int64

Initial raw data:

,tweet_id,from_platform,sentiment,tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


Initial raw data shape:
(74682, 4)

Complete processing of all features: handling missing values, outliers, scaling, vectorize text colums, encoding categorials features, etc..

In [2]:
## 3. Data Preprocessing and Feature Engineering
if raw_data_df is not None:

    # Complete processing of all features: handling missing values, outliers, scaling, vectorize text colums, encoding categorials features, etc..
    # Process training data  + save it in the processed directory
    data_preprocessed_df = data_processor.process_data(raw_data_df, raw_data_file_name)
    
    if data_preprocessed_df is not None:
        print("Training data preprocessing completed successfully")
        display(data_preprocessed_df.head())
        
        # PREPARE AND PROCESS TEST DATA
        print('\nPROCESSING TEST DATA')
        raw_data_test_file_name = config['data']['raw_data_test_file_name']

        if raw_data_test_file_name:
            # Case 1: Separate test file exists - process it independently
            print(f"\nProcessing separate test file: {raw_data_test_file_name}")
            raw_data_test_df = data_processor.data_load(f'{raw_data_path}/{raw_data_test_file_name}')
            
            if raw_data_test_df is not None:
                # Process test data using same processor to maintain consistency + save it in the processed directory
                data_test_preprocessed_df = data_processor.process_data(raw_data_test_df, raw_data_test_file_name)
                
                if data_test_preprocessed_df is not None:
                    print("Test data preprocessing completed successfully")
                    display('\ndata_test_preprocessed_df: ', data_test_preprocessed_df.head())
                else:
                    print('Test data preprocessing failed')
            else:
                print('Test data loading failed')    
        else:
            # Case 2: Create test data from training data
            print("\nNo separate test file. Creating test data from training data...")
            
            # Get test size from config
            test_size = config.get('model', {}).get('params', {}).get('test_size', 0.2)
            
            # Split the processed data
            from sklearn.model_selection import train_test_split
            train_df, test_df = train_test_split(
                data_preprocessed_df,
                test_size=test_size,
                random_state=config.get('model', {}).get('params', {}).get('random_state', 42)
            )
            
            # Save split datasets
            print("\nSaving split datasets...")
            
            # Save training portion
            train_file_name = raw_data_file_name
            data_processor.save_processed_data(train_df, train_file_name) # this will update the processed raw data file already saved
            print(f"Training data saved successfully. Updated the processed raw data file already saved ")
            
            # Save test portion
            import os
            file_name, file_extension = os.path.splitext(raw_data_file_name)
            test_file_name = f"{file_name}_test{file_extension}"
            data_processor.save_processed_data(test_df, test_file_name)
            print(f"Test data saved successfully")
            
            # Update preprocessed dataframes
            data_preprocessed_df = train_df
            data_test_preprocessed_df = test_df
            
            print(f"\nFinal shapes:")
            print(f"Training data: {train_df.shape}")
            print(f"Test data: {test_df.shape}")
    else:
        print('Training data preprocessing failed')
else:
    print('Data loading failed')

DATA PROCESS: missing values in numeric columns
 tweet_id    0
dtype: int64

DATA PROCESS: Missing values in numeric columns after treatment 
 tweet_id    0
dtype: int64

DATA PROCESS: categorical_columns
 Index(['from_platform', 'sentiment', 'tweet'], dtype='object')

DATA PROCESS: Data after handling missing values
        tweet_id from_platform sentiment  \
0          2401   Borderlands  Positive   
1          2401   Borderlands  Positive   
2          2401   Borderlands  Positive   
3          2401   Borderlands  Positive   
4          2401   Borderlands  Positive   
...         ...           ...       ...   
74677      9200        Nvidia  Positive   
74678      9200        Nvidia  Positive   
74679      9200        Nvidia  Positive   
74680      9200        Nvidia  Positive   
74681      9200        Nvidia  Positive   

                                                   tweet  
0      im getting on borderlands and i will murder yo...  
1      I am coming to the borders and I will kill you...  
2      im getting on borderlands and i will kill you ...  
3      im coming on borderlands and i will murder you...  
4      im getting on borderlands 2 and i will murder ...  
...                                                  ...  
74677  Just realized that the Windows partition of my...  
74678  Just realized that my Mac window partition is ...  
74679  Just realized the windows partition of my Mac ...  
74680  Just realized between the windows partition of...  
74681  Just like the windows partition of my Mac is l...  

[74682 rows x 4 columns]

DATA PROCESS: row_data_column_id_index to drop:
 -1

...

DATA PROCESS: Saving processed data to a file

Test data saved successfully

Final shapes:

Training data: (59745, 53)

Test data: (14937, 53)

PREPARE VALIDATION DATA: LOAD AND PROCESS RAW DATA TEST FILE IF EXIST or SPLIT DATA PROCESSED IN TWO PARTS (one for the train another for the test)

In [4]:
## 3. Data Preprocessing and Feature Engineering
if raw_data_df is not None:
    # Process training data
    data_preprocessed_df = data_processor.process_data(raw_data_df, raw_data_file_name)
    
    if data_preprocessed_df is not None:
        print("Training data preprocessing completed successfully")
        display(data_preprocessed_df.head())
        
        # PREPARE AND PROCESS TEST DATA
        print('\nPROCESSING TEST DATA')
        raw_data_test_file_name = config['data']['raw_data_test_file_name']

        if raw_data_test_file_name:
            # Case 1: Separate test file exists - process it independently
            print(f"\nProcessing separate test file: {raw_data_test_file_name}")
            raw_data_test_df = data_processor.data_load(f'{raw_data_path}/{raw_data_test_file_name}')
            
            if raw_data_test_df is not None:
                # Process test data using same processor to maintain consistency
                data_test_preprocessed_df = data_processor.process_data(raw_data_test_df, raw_data_test_file_name)
                
                if data_test_preprocessed_df is not None:
                    print("Test data preprocessing completed successfully")
                    display(data_test_preprocessed_df.head())
                else:
                    print('Test data preprocessing failed')
            else:
                print('Test data loading failed')    
        else:
            # Case 2: Create test data from training data
            print("\nNo separate test file. Creating test data from training data...")
            
            # Get test size from config
            test_size = config.get('model', {}).get('params', {}).get('test_size', 0.2)
            
            # Split the processed data
            from sklearn.model_selection import train_test_split
            train_df, test_df = train_test_split(
                data_preprocessed_df,
                test_size=test_size,
                random_state=config.get('model', {}).get('params', {}).get('random_state', 42)
            )
            
            # Save split datasets
            print("\nSaving split datasets...")
            
            # Save training portion
            train_file_name = raw_data_file_name
            data_processor.save_processed_data(train_df, train_file_name)
            print(f"Training data saved successfully")
            
            # Save test portion
            import os
            file_name, file_extension = os.path.splitext(raw_data_file_name)
            test_file_name = f"{file_name}_test{file_extension}"
            data_processor.save_processed_data(test_df, test_file_name)
            print(f"Test data saved successfully")
            
            # Update preprocessed dataframes
            data_preprocessed_df = train_df
            data_test_preprocessed_df = test_df
            
            print(f"\nFinal shapes:")
            print(f"Training data: {train_df.shape}")
            print(f"Test data: {test_df.shape}")
    else:
        print('Training data preprocessing failed')
else:
    print('Data loading failed')

DATA PROCESS: missing values in numeric columns
 tweet_id    0
dtype: int64

DATA PROCESS: Missing values in numeric columns after treatment 
 tweet_id    0
dtype: int64

DATA PROCESS: categorical_columns
 Index(['from_platform', 'sentiment', 'tweet'], dtype='object')

DATA PROCESS: Data after handling missing values
        tweet_id from_platform sentiment  \
0          2401   Borderlands  Positive   
1          2401   Borderlands  Positive   
2          2401   Borderlands  Positive   
3          2401   Borderlands  Positive   
4          2401   Borderlands  Positive   
...         ...           ...       ...   
74677      9200        Nvidia  Positive   
74678      9200        Nvidia  Positive   
74679      9200        Nvidia  Positive   
74680      9200        Nvidia  Positive   
74681      9200        Nvidia  Positive   

                                                   tweet  
0      im getting on borderlands and i will murder yo...  
1      I am coming to the borders and I will kill you...  
2      im getting on borderlands and i will kill you ...  
3      im coming on borderlands and i will murder you...  
4      im getting on borderlands 2 and i will murder ...  
...                                                  ...  
74677  Just realized that the Windows partition of my...  
74678  Just realized that my Mac window partition is ...  
74679  Just realized the windows partition of my Mac ...  
74680  Just realized between the windows partition of...  
74681  Just like the windows partition of my Mac is l...  

[74682 rows x 4 columns]

DATA PROCESS: row_data_column_id_index to drop:
 -1

...

DATA PROCESS: Saving processed data to a file

Test data saved successfully

Final shapes:

Training data: (59745, 53)

Test data: (14937, 53)

In [ ]:
## 4. Data Analysis and Insights

# Initialize analyzer
data_analyser = DataAnalyzer(data_preprocessed_df)

# Generate and display summary
summary = data_analyser.generate_summary()
print('\nData summary after process and analysis:')
print(summary)

# Plot correlations
data_analyser.plot_correlations()

JUST FOR TEST

In [ ]:
np.random.seed = (123)
mu, sigma = 5, 1
x_train_1_1 = np.random.normal(mu,sigma,10)
x_train_1_2 = np.random.normal(2,9,10)

x_train_2_1 = np.random.normal(11,18,10)
x_train_2_2 = np.random.normal(19,22,10)

x_train_3_1 = np.random.normal(25,29,10)
x_train_3_2 = np.random.normal(30,43,10)

print('\n x_train_1_1: ', x_train_1_1)
print('\n x_train_2_2: ', x_train_2_2)

print('\n concate de x_train_1_1, x_train_2_2: ', np.concatenate([x_train_1_1, x_train_2_2]))

x_train = np.stack([
                   np.concatenate([x_train_1_1, x_train_2_1, x_train_3_1]),
                   np.concatenate([x_train_1_2, x_train_2_2, x_train_3_2]),
                   ],
                   axis=1)

x_train

In [5]:
## 5. Model Training Preparation

# Prepare features and target
target_column_name = data_processor.config['data']['raw_data_target_column']['name']
target_column_type = data_processor.config['data']['raw_data_target_column']['type']

if target_column_type == 'categorical':
    target_column_name = f'{target_column_name}_categorical_encoded' # By convention, the data process phase add the word "_categorical_encoded" to each encoded categorical feature

if target_column_name in data_preprocessed_df.columns:
    X = data_preprocessed_df.drop(columns=[target_column_name])
    y = data_preprocessed_df[target_column_name]
else:
    X = data_preprocessed_df
    y = None

print("Features shape:", X.shape)
if y is not None:
    print("Target shape:", y.shape)
else:
    print('No target (or valid target) defined in the conf file -> Non supervised learning will be applied !')

Features shape:
(59745, 52)

Target shape:
(59745,)

In [ ]:
## 6. Model Training and Evaluation

#print('Start training')
# Initialize trainer
#trainer = ModelTrainer(data_processor.config)

# Train model
#metrics = trainer.train_model(X, y)
#print("\nTraining metrics:")
#print(metrics)

# Cross-validation if applicable
#if y is not None:
#    cv_results = trainer.cross_validate(X, y)
#    print("\nCross-validation results:")
#    print(cv_results)

# Feature importance
#importance = trainer.get_feature_importance()
#if importance:
#    import matplotlib.pyplot as plt
#    plt.figure(figsize=(10, 6))
#    features = list(importance.keys())
#    values = list(importance.values())
#    plt.barh(features, values)
#    plt.title('Feature Importance')
#    plt.xlabel('Importance')
#    plt.show()

In [ ]:
## 7. Model Saving

# Save model and history
#model_name = config['model']['name']
#model_version = config['model']['version']
#save_path = f"../models/{model_name}_{model_version}"
#trainer.save_model(save_path)
#print(f"Model saved to {save_path}")